In [1]:
from sdRDM.generator import generate_python_api
import numpy as np

In [2]:
# generate_python_api("specifications/photometry.md", "", "PlateHandler")

In [3]:
%reload_ext autoreload
%autoreload 2

from PlateHandler.modified.plate import Plate

# Load time-course data and create a ```Plate``` object


In [4]:
path = "/Users/max/Documents/enzymeml/alaric/EnzymeML_SLAC_5xConcentration_pH2.0-3.5/ABTS_EnzymeML_340nm_420nm_5x_35deg_pH3.5.txt"

time = np.linspace(0, 15, 31)

plate = Plate.from_file(path, time, "min", ph=3.5, temperature=35)

In [5]:
# print(plate)

# Define Species, that are present across different wells

Each time a ```Well``` is created, a ```Species``` of the type 'buffer' is added to it.

In [6]:
buffer = plate.add_reactant(id="s1", name="buffer", constant=True)
abts = plate.add_reactant(id="s0", name="ABTS", constant=False,
                          inchi="InChI=1S/C18H18N4O6S4/c1-3-21-13-7-5-11(31(23,24)25)9-15(13)29-17(21)19-20-18-22(4-2)14-8-6-12(32(26,27)28)10-16(14)30-18/h5-10H,3-4H2,1-2H3,(H,23,24,25)(H,26,27,28)/b19-17-,20-18+")
slac = plate.add_protein(id="p0", name="SLAC",
                         constant=True, sequence="MSSKSKPKDVKV")

In [7]:
plate.species

[Reactant(id='s1', name='buffer', vessel_id='plate0', init_conc=None, constant=True, unit=None, uri=None, creator_id=None, smiles=None, inchi=None, chebi_id=None, ontology='SBO:0000247'),
 Reactant(id='s0', name='ABTS', vessel_id='plate0', init_conc=None, constant=False, unit=None, uri=None, creator_id=None, smiles=None, inchi='InChI=1S/C18H18N4O6S4/c1-3-21-13-7-5-11(31(23,24)25)9-15(13)29-17(21)19-20-18-22(4-2)14-8-6-12(32(26,27)28)10-16(14)30-18/h5-10H,3-4H2,1-2H3,(H,23,24,25)(H,26,27,28)/b19-17-,20-18+', chebi_id=None, ontology='SBO:0000247'),
 Protein(id='p0', name='SLAC', vessel_id='plate0', init_conc=None, constant=True, unit=None, uri=None, creator_id=None, sequence='MSSKSKPKDVKV', ecnumber=None, organism=None, organism_tax_id=None, uniprotid=None, ontology='SBO:0000013')]

### Assign initial concentrations to different wells

In order to blank the data, we need to assign initial concentrations to the wells. Currently, initial conditions can be added per species, whereas different concentrations can be assigned to an entire row. If a s species is not present in the specified row, its concentration should be set to zero.

In [8]:
print(buffer)

Reactant
├── id = s1
├── name = buffer
├── vessel_id = plate0
├── constant = True
└── ontology = SBO:0000247



In [9]:
# Define buffer concentration
plate.assign_species_conditions_to_rows(
    row_ids=["A", "B", "C", "D", "E", "F"],
    species=buffer,
    init_concs=100,
    conc_unit="mM"
)

# Define substrate concentrations
plate.assign_species_conditions_to_rows(
    row_ids=["A", "B", "C", "D", "E", "F"],
    species=abts,
    init_concs=[0, 5, 10, 15, 25, 50, 75, 100, 150, 200],
    conc_unit="mM"
)

# Define enzyme concentrations
plate.assign_species_conditions_to_rows(
    row_ids=["A", "B", "C"],
    species=slac,
    init_concs=4,
    conc_unit="nM"
)

In [10]:
print(plate.wells[6])

Well
├── id = A7
├── absorption = [1.35592973769869, 1.27878673625631, 1.22387991465835, 1.18880050168404, 1.13650156496561, ...]
├── time = [0.0, 0.5, 1.0, 1.5, 2.0, ...]
├── time_unit = min
├── init_conditions
│   ├── 0
│   │   └── InitCondition
│   │       ├── id = initcondition12
│   │       ├── species_id = s1
│   │       ├── init_conc = 100.0
│   │       ├── conc_unit = mM
│   │       └── was_blanked = False
│   ├── 1
│   │   └── InitCondition
│   │       ├── id = initcondition130
│   │       ├── species_id = s0
│   │       ├── init_conc = 75.0
│   │       ├── conc_unit = mM
│   │       └── was_blanked = False
│   └── 2
│       └── InitCondition
│           ├── id = initcondition240
│           ├── species_id = p0
│           ├── init_conc = 4.0
│           ├── conc_unit = nM
│           └── was_blanked = False
├── x_position = 6
├── y_position = 0
└── wavelength = 340



### Blank data

Since we have assigned the initial concentrations for each species, blanking id carried out semi-automatically. By calling ```blank_species()```, the absorption contribution of the defined species is subtracted from all wells, in which the species is present.

__How does it work?__  
After specifying the concentration of each species present in each well for each well respectively, wells which were prepared for blanking are be implicitly identified.
If we call the blank function and specify a species which should be blanked, the blanking function first identifies wells which solely contain the specified species. Thereafter, the mean value across all identified wells for blanking is calculated. Thereafter, all wells which contain the selected species are selected and the calculated blank is subtracted from all absorption values. Lastly, a flag (```was_blanked = True```) is set, that prevents the species from being used for blanking again

In [12]:
plate.blank_species(buffer, 340)
plate.blank_species(slac, 340)

In [14]:
print(plate.wells[6])

Well
├── id = A7
├── absorption = [1.20710280045204, 1.12995979900966, 1.0750529774117001, 1.03997356443739, 0.9876746277189601, ...]
├── time = [0.0, 0.5, 1.0, 1.5, 2.0, ...]
├── time_unit = min
├── init_conditions
│   ├── 0
│   │   └── InitCondition
│   │       ├── id = initcondition12
│   │       ├── species_id = s1
│   │       ├── init_conc = 100.0
│   │       ├── conc_unit = mM
│   │       └── was_blanked = True
│   ├── 1
│   │   └── InitCondition
│   │       ├── id = initcondition130
│   │       ├── species_id = s0
│   │       ├── init_conc = 75.0
│   │       ├── conc_unit = mM
│   │       └── was_blanked = False
│   └── 2
│       └── InitCondition
│           ├── id = initcondition240
│           ├── species_id = p0
│           ├── init_conc = 4.0
│           ├── conc_unit = nM
│           └── was_blanked = True
├── x_position = 6
├── y_position = 0
└── wavelength = 340

